# Knapsack problem

In [8]:
%load_ext autoreload
%autoreload 2

In [1]:
from src import qubo, data, ising, brute_force
import numpy as np

## Select the problem

In [2]:
kp_type = 'small'
problems = data.load(kp_type, path='kp_instances')
problem = problems[0]
problem

{'C': 10,
 'n': 8,
 'weights': [6, 6, 7, 4, 3, 3, 2, 2],
 'profits': [6, 7, 7, 4, 3, 4, 2, 1]}

## Brute force

In [3]:
bf_sol = None
if kp_type == 'large':
    print('I CANNOT DO THAT SIR!')
else:
    max_ram = 40E9
    bf_sol = brute_force.kp_brute_force(problem['profits'], problem['weights'], problem['C'], max_ram)
bf_sol

Number of threads: 15
Max samples: 31250000
Number of steps: 1


100%|██████████| 1/1 [00:00<00:00, 53.55it/s]


{'profit': array([11, 11, 11, 11]),
 'cost': array([10, 10,  9, 10]),
 'combo': [array([3, 4, 5]), array([2, 5]), array([1, 5]), array([1, 3])]}

In [4]:
# get the solution which also minimises the cost
best_indices = np.flatnonzero(bf_sol['cost'] == bf_sol['cost'].min())

print('Best solution(s) found by brute force:')
print('Profit:', bf_sol['profit'][best_indices[0]])
print('Cost:', bf_sol['cost'][best_indices[0]])
print('Combo(s):', [bf_sol['combo'][i] for i in best_indices])

Best solution(s) found by brute force:
Profit: 11
Cost: 9
Combo(s): [array([1, 5])]


## Exact diagonalization

In [28]:
# get the qubo matrix
LAMBDA = max(problem['profits']) + 2
Q = qubo.get_Q(problem['weights'], problem['profits'], problem['C'], LAMBDA)

# embed the qubo matrix into an hamiltonian
h, J, offset = ising.qubo_to_hamiltonian(Q)

if kp_type != 'small':
    print('I CANNOT DO THAT SIR!')
else:
    # get the diagonal of the hamiltonian
    diag = ising.construct_quantum_hamiltonian_scipy(h, J, offset)

    diag_sol = np.argmin(diag)
    diag_sol_bits = (~brute_force.toBitstrings(diag_sol, Q.shape[0])).astype(int)
    cost = diag_sol_bits[:problem['n']].dot(problem['weights'])
    profit = diag_sol_bits[:problem['n']].dot(problem['profits'])
    print('Solution found by diagonalising the Hamiltonian:')
    print('Profit:', profit)
    print('Cost:', cost, f" (but cost was: {problem['C']}" if cost > problem['C'] else '')
    print('Combo:', diag_sol_bits[:problem['n']])

Solution found by diagonalising the Hamiltonian:
Profit: 10
Cost: 10 
Combo: [0 0 0 0 1 1 1 1]


## QAOA in Qiskit

In [57]:
from qiskit.result import QuasiDistribution

from qiskit_algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA

from qiskit_algorithms.utils import algorithm_globals
from qiskit.circuit.library import QAOAAnsatz
from qiskit.primitives import Sampler
from qiskit.quantum_info import Statevector

def calc_cost_and_constraint(profits, weights, solution, C):
    assert len(profits) == len(weights)
    s = len(profits)

    total_cost = sum(profits[i] * solution[i] for i in range(s))
    print("Optimum value of profit function from ED = ",total_cost)

    slack = sum((2**i) * solution[s + i] for i in range(len(solution) - (s + 1)))
    slack += (C + 1 - 2**(len(solution) - s - 1)) * solution[-1]
    print("Total value of slack variable = ", slack)

    total_weight = sum(weights[k] * solution[k] for k in range(s))
    if (total_weight <= C):
       print("Total weight less than given constraint")
    else:
       print("Weight constraint not satisfied; wrong solution")

In [31]:
# get the qubo matrix
LAMBDA = max(problem['profits']) + 2
Q = qubo.get_Q(problem['weights'], problem['profits'], problem['C'], LAMBDA)

# embed the qubo matrix into an hamiltonian
h, J, offset = ising.qubo_to_hamiltonian(Q)

H = ising.construct_quantum_hamiltonian_qiskit(h, J, offset)

In [60]:
sampler = Sampler()
algorithm_globals.random_seed = 10598

optimizer = COBYLA()
qaoa = QAOA(sampler, optimizer, reps=2)

result = qaoa.compute_minimum_eigenvalue(H)

In [59]:
# get the solution
qaoa_qk_sol = result.best_measurement['bitstring']
qaoa_qk_sol_bits = np.logical_not(np.array([int(bit) for bit in qaoa_qk_sol])).astype(int)
print('Solution found by QAOA:')
print('Profit:', qaoa_qk_sol_bits[:problem['n']].dot(problem['profits']))
print('Cost:', qaoa_qk_sol_bits[:problem['n']].dot(problem['weights']))
print('Combo:', qaoa_qk_sol_bits[:problem['n']])
print('Slack:', qaoa_qk_sol_bits[problem['n']:])
calc_cost_and_constraint(problem['profits'], problem['weights'], qaoa_qk_sol_bits, problem['C'])

Solution found by QAOA:
Profit: 10
Cost: 10
Combo: [0 0 0 0 1 1 1 1]
Slack: [0 0 0 0]
Optimum value of profit function from ED =  10
Total value of slack variable =  0
Total weight less than given constraint


## QAOA in Matcha Tea